In [ ]:
import h5py
import pandas as pd
import numpy as np

In [ ]:
f = h5py.File('raw/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth_0.z', 'r')
treeArray = f['t_allpar_new'][()]
f.close()
print(treeArray.dtype.names)

In [ ]:
features = ["j1_e","j1_px","j1_py","j1_pz"]
j_index = ["j_index"]
labels = ['j_q', 'j_g', 'j_w', 'j_z', 'j_t']

In [ ]:
df = pd.DataFrame(treeArray,columns=list(set(features+j_index)))
df = df.drop_duplicates()
df = df[features+j_index]

In [ ]:
df2 = (df.pivot_table(index = "j_index", columns = df.groupby("j_index").cumcount().add(1), aggfunc = "first", fill_value = 0).sort_index(axis = 1, level = 1))
df2 = (df2.set_axis([f'{x}{y}' for x,y in df2.columns], axis=1).reset_index())
df2 = df2.iloc[:, 1:801]
df2.reset_index(drop = True, inplace = True)

In [ ]:
df2

In [ ]:
j_px = treeArray["j_pt"]
j_py = np.zeros((treeArray["index"].size))
j_pz = treeArray["j_pt"] * np.sinh(treeArray["j_eta"])
j_E = np.sqrt(np.square(treeArray["j_mass"]) + np.square(treeArray["j_pt"]) + np.square(j_pz))
features_np = np.stack((j_E, j_px, j_py, j_pz, treeArray["j_q"], treeArray["j_g"], treeArray["j_w"], treeArray["j_z"], treeArray["j_t"], treeArray["j_index"]))

In [ ]:
features_np = features_np.T

In [ ]:
features_df = pd.DataFrame(features_np, columns = ["j_E", "j_px", "j_py", "j_pz", "j_q", "j_g", "j_w", "j_z", "j_t", "j_index"])
features_df = features_df.drop_duplicates()
features_df.sort_values(by=["j_index"], inplace=True)
features_df = features_df.iloc[:, :-1]
features_df.reset_index(drop = True, inplace = True)

In [ ]:
features_df

In [ ]:
df2 = df2.append(features_df)

In [ ]:
df2